# CS2 Inter-Market Arbitrage Analysis

This notebook performs exploratory data analysis, identifies arbitrage opportunities, and evaluates risk metrics.


In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import sqlite3

from src.db.client import DatabaseClient
from src.analysis.risk import RiskAnalyzer

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)


## 1. Load Data from Database


In [ ]:
# Initialize database client
db_client = DatabaseClient('db/arbitrage.sqlite')

# Get latest snapshots
snapshots = db_client.get_latest_snapshots()

print(f"Loaded {len(snapshots)} item snapshots")

# Convert to DataFrame
df_snapshots = pd.DataFrame(snapshots)
df_snapshots.head()


## 2. Calculate PnL and Derived Metrics


In [ ]:
# Initialize risk analyzer
risk_analyzer = RiskAnalyzer(tc_steam=0.15)

# Filter out items with missing data
df_clean = df_snapshots[
    df_snapshots['steam_best_bid'].notna() & 
    df_snapshots['buff_best_ask'].notna()
].copy()

# Calculate metrics
df_clean['adj_steam_bid'] = df_clean['steam_best_bid'] * (1 - 0.15)
df_clean['pnl_now'] = df_clean['adj_steam_bid'] - df_clean['buff_best_ask']
df_clean['spread_pct'] = (df_clean['pnl_now'] / df_clean['buff_best_ask']) * 100

# Filter positive PnL
df_positive = df_clean[df_clean['pnl_now'] > 0].copy()

print(f"Items with positive PnL: {len(df_positive)} / {len(df_clean)}")
print(f"\nTop 10 by PnL:")
df_positive.nlargest(10, 'pnl_now')[['market_hash_name', 'steam_best_bid', 'buff_best_ask', 'pnl_now', 'spread_pct']]


## 2.5. Spread Distribution Visualization


In [ ]:
# Plot spread distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of spreads
axes[0].hist(df_clean['spread_pct'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='r', linestyle='--', label='Break-even')
axes[0].set_xlabel('Spread %')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Spreads (All Items)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot: positive vs negative spreads
df_clean['spread_category'] = df_clean['spread_pct'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
df_clean.boxplot(column='spread_pct', by='spread_category', ax=axes[1])
axes[1].set_xlabel('Spread Category')
axes[1].set_ylabel('Spread %')
axes[1].set_title('Spread Distribution by Category')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary stats
print(f"Mean spread: {df_clean['spread_pct'].mean():.2f}%")
print(f"Median spread: {df_clean['spread_pct'].median():.2f}%")
print(f"Std dev: {df_clean['spread_pct'].std():.2f}%")


## 3. Price History and Volatility Analysis


In [ ]:
# Get price history for top candidates
top_items = df_positive.nlargest(5, 'pnl_now')

volatility_data = []

for _, row in top_items.iterrows():
    item_id = row['item_id']
    price_history = db_client.get_price_history(item_id, days=7)
    
    steam_prices = [
        float(s.get('best_bid', 0) or s.get('median_price', 0))
        for s in price_history.get('steam', [])
        if s.get('best_bid') or s.get('median_price')
    ]
    
    if len(steam_prices) >= 2:
        volatility = risk_analyzer.calculate_volatility(steam_prices)
        volatility_data.append({
            'item_id': item_id,
            'market_hash_name': row['market_hash_name'],
            'volatility': volatility,
            'price_observations': len(steam_prices),
            'current_pnl': row['pnl_now']
        })

df_volatility = pd.DataFrame(volatility_data)
print("Volatility Analysis:")
df_volatility


## 3.5. Price History Time Series


In [ ]:
# Plot price history for top candidates
if len(df_positive) > 0:
    top_item = df_positive.nlargest(1, 'pnl_now').iloc[0]
    item_id = top_item['item_id']
    item_name = top_item['market_hash_name']
    
    price_history = db_client.get_price_history(item_id, days=7)
    
    # Prepare data
    steam_data = price_history.get('steam', [])
    buff_data = price_history.get('buff', [])
    
    if steam_data and buff_data:
        # Convert to DataFrames
        steam_df = pd.DataFrame(steam_data)
        buff_df = pd.DataFrame(buff_data)
        
        # Parse timestamps
        steam_df['timestamp'] = pd.to_datetime(steam_df['timestamp'])
        buff_df['timestamp'] = pd.to_datetime(buff_df['timestamp'])
        
        # Get prices
        steam_df['price'] = steam_df['best_bid'].fillna(steam_df.get('median_price', 0))
        buff_df['price'] = buff_df['best_ask']
        
        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(steam_df['timestamp'], steam_df['price'], marker='o', label='Steam (bid)', alpha=0.7)
        plt.plot(buff_df['timestamp'], buff_df['price'], marker='s', label='Buff (ask)', alpha=0.7)
        
        # Add adjusted Steam line (after fee)
        if 'price' in steam_df.columns:
            steam_df['adj_price'] = steam_df['price'] * 0.85
            plt.plot(steam_df['timestamp'], steam_df['adj_price'], 
                    marker='x', label='Steam (adj for 15% fee)', linestyle='--', alpha=0.7)
        
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'Price History: {item_name[:50]}')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print(f"Insufficient price history for {item_name}")


## 4. Risk Analysis with Monte Carlo Simulation


## 4.5. Risk Metrics Visualization


In [ ]:
# Risk metrics plots
if len(df_risk) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Probability of positive PnL distribution
    axes[0, 0].hist(df_risk['prob_positive'], bins=30, edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(x=0.5, color='r', linestyle='--', label='50% threshold')
    axes[0, 0].set_xlabel('Probability of Positive PnL')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Distribution of P(Positive PnL)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Expected PnL vs Current PnL
    axes[0, 1].scatter(df_risk['pnl_now'], df_risk['expected_pnl'], alpha=0.6)
    axes[0, 1].plot([df_risk['pnl_now'].min(), df_risk['pnl_now'].max()], 
                    [df_risk['pnl_now'].min(), df_risk['pnl_now'].max()], 
                    'r--', label='y=x (no change)')
    axes[0, 1].set_xlabel('Current PnL')
    axes[0, 1].set_ylabel('Expected PnL after hold')
    axes[0, 1].set_title('Current vs Expected PnL')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. VaR distribution
    axes[1, 0].hist(df_risk['var_95'], bins=30, edgecolor='black', alpha=0.7)
    axes[1, 0].axvline(x=0, color='r', linestyle='--', label='Break-even')
    axes[1, 0].set_xlabel('95% VaR (loss)')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Distribution of 95% Value at Risk')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Risk-return scatter
    axes[1, 1].scatter(df_risk['var_95'], df_risk['expected_pnl'], 
                       c=df_risk['prob_positive'], cmap='RdYlGn', alpha=0.6, s=50)
    axes[1, 1].set_xlabel('95% VaR (risk)')
    axes[1, 1].set_ylabel('Expected PnL (return)')
    axes[1, 1].set_title('Risk-Return Profile (color = P(positive))')
    cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
    cbar.set_label('P(Positive PnL)')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Analyze hold period risk for top candidates
hold_days = 3
risk_results = []

for _, row in df_positive.nlargest(10, 'pnl_now').iterrows():
    steam_bid = float(row['steam_best_bid'])
    buff_ask = float(row['buff_best_ask'])
    
    # Get volatility
    item_id = row['item_id']
    price_history = db_client.get_price_history(item_id, days=7)
    steam_prices = [
        float(s.get('best_bid', 0) or s.get('median_price', 0))
        for s in price_history.get('steam', [])
        if s.get('best_bid') or s.get('median_price')
    ]
    
    if len(steam_prices) < 2:
        volatility = 0.10  # Default 10%
    else:
        volatility = risk_analyzer.calculate_volatility(steam_prices)
    
    # Risk analysis
    risk_metrics = risk_analyzer.analyze_hold_period_risk(
        steam_bid, buff_ask, volatility, hold_days
    )
    
    risk_results.append({
        'item_id': item_id,
        'market_hash_name': row['market_hash_name'],
        'pnl_now': row['pnl_now'],
        'volatility': volatility,
        **risk_metrics
    })

df_risk = pd.DataFrame(risk_results)
print("Risk Analysis Results:")
df_risk[['market_hash_name', 'pnl_now', 'volatility', 'prob_positive', 'expected_pnl', 'var_95']]


## 5. Visualizations


## 5.5. Correlation Analysis


In [ ]:
# Correlation matrix for key metrics
if len(df_risk) > 0:
    # Combine data
    df_corr = df_risk[['pnl_now', 'spread_pct', 'volatility', 'prob_positive', 
                       'expected_pnl', 'var_95']].copy()
    
    # Calculate correlation
    corr_matrix = df_corr.corr()
    
    # Plot heatmap
    plt.figure(figsize=(10, 8))
    im = plt.imshow(corr_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
    plt.colorbar(im, label='Correlation')
    
    # Add text annotations
    for i in range(len(corr_matrix.columns)):
        for j in range(len(corr_matrix.columns)):
            text = plt.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                           ha="center", va="center", color="black", fontsize=9)
    
    plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=45, ha='right')
    plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns)
    plt.title('Correlation Matrix: Key Metrics')
    plt.tight_layout()
    plt.show()
    
    print("\nKey Correlations:")
    print(f"Spread vs Expected PnL: {df_corr['spread_pct'].corr(df_corr['expected_pnl']):.3f}")
    print(f"Volatility vs VaR: {df_corr['volatility'].corr(df_corr['var_95']):.3f}")
    print(f"Expected PnL vs P(Positive): {df_corr['expected_pnl'].corr(df_corr['prob_positive']):.3f}")


In [ ]:
# Histogram of PnL
plt.figure(figsize=(10, 6))
plt.hist(df_positive['pnl_now'], bins=30, edgecolor='black')
plt.xlabel('PnL (currency units)')
plt.ylabel('Frequency')
plt.title('Distribution of Current PnL')
plt.grid(True, alpha=0.3)
plt.show()


## 7.5. Ask-Only Pressure: Additional Analysis


In [ ]:
# Additional analysis for ask-only pressure theory
if len(df_imbalance) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Ask/Bid ratio distribution
    axes[0, 0].hist(df_imbalance['imbalance_ratio'], bins=50, edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(x=1, color='r', linestyle='--', label='Balanced (1:1)')
    axes[0, 0].set_xlabel('Ask/Bid Ratio')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Distribution of Ask/Bid Ratios')
    axes[0, 0].set_xlim(0, min(10, df_imbalance['imbalance_ratio'].quantile(0.95)))
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Ask count vs Bid count scatter
    axes[0, 1].scatter(df_imbalance['bid_count'], df_imbalance['ask_count'], alpha=0.6)
    max_val = max(df_imbalance['ask_count'].max(), df_imbalance['bid_count'].max())
    axes[0, 1].plot([0, max_val], [0, max_val], 'r--', label='Equal asks/bids')
    axes[0, 1].set_xlabel('Bid Count')
    axes[0, 1].set_ylabel('Ask Count')
    axes[0, 1].set_title('Ask vs Bid Counts')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Imbalance ratio vs Spread (with trend line)
    axes[1, 0].scatter(df_imbalance['imbalance_ratio'], df_imbalance['spread_pct'], alpha=0.6)
    # Add trend line
    z = np.polyfit(df_imbalance['imbalance_ratio'], df_imbalance['spread_pct'], 1)
    p = np.poly1d(z)
    x_trend = np.linspace(df_imbalance['imbalance_ratio'].min(), 
                         df_imbalance['imbalance_ratio'].max(), 100)
    axes[1, 0].plot(x_trend, p(x_trend), "r--", alpha=0.8, label=f'Trend: y={z[0]:.2f}x+{z[1]:.2f}')
    axes[1, 0].set_xlabel('Ask/Bid Ratio')
    axes[1, 0].set_ylabel('Spread %')
    axes[1, 0].set_title('Imbalance Ratio vs Spread (with trend)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Box plot: spread by imbalance category
    df_imbalance['imbalance_category'] = df_imbalance['imbalance_ratio'].apply(
        lambda x: 'Ask-heavy (>2:1)' if x > 2 else ('Balanced (0.5-2)' if x > 0.5 else 'Bid-heavy (<0.5:1)')
    )
    df_imbalance.boxplot(column='spread_pct', by='imbalance_category', ax=axes[1, 1])
    axes[1, 1].set_xlabel('Imbalance Category')
    axes[1, 1].set_ylabel('Spread %')
    axes[1, 1].set_title('Spread by Imbalance Category')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistical test
    ask_heavy = df_imbalance[df_imbalance['imbalance_ratio'] > 2]['spread_pct']
    balanced = df_imbalance[(df_imbalance['imbalance_ratio'] >= 0.5) & 
                           (df_imbalance['imbalance_ratio'] <= 2)]['spread_pct']
    
    if len(ask_heavy) > 0 and len(balanced) > 0:
        print(f"\nAsk-heavy items (>2:1): {len(ask_heavy)} items, mean spread: {ask_heavy.mean():.2f}%")
        print(f"Balanced items (0.5-2:1): {len(balanced)} items, mean spread: {balanced.mean():.2f}%")


In [ ]:
# Scatter: Volatility vs Expected PnL
if len(df_risk) > 0:
    plt.figure(figsize=(10, 6))
    plt.scatter(df_risk['volatility'], df_risk['expected_pnl'], alpha=0.6)
    plt.xlabel('Volatility (daily)')
    plt.ylabel('Expected PnL after hold')
    plt.title('Volatility vs Expected PnL (3-day hold)')
    plt.grid(True, alpha=0.3)
    plt.show()


In [ ]:
# Top candidates table
if len(df_risk) > 0:
    df_top = df_risk.nlargest(10, 'expected_pnl')
    print("Top 10 Candidates by Expected PnL:")
    print(df_top[['market_hash_name', 'pnl_now', 'expected_pnl', 'prob_positive', 'var_95']].to_string(index=False))


## 7. Buff Ask-Only Pressure Analysis

Testing theory: Steam's lack of direct cashouts vs Buff's cashout capability creates ask-only pressure on Buff (sellers want to exit, fewer buyers).


In [ ]:
# Analyze Buff order book imbalance
# Theory: More asks than bids = sellers want to exit (cashout pressure)
# Compare ask/bid depth and order counts

buff_imbalance_data = []

for _, row in df_clean.iterrows():
    item_id = row['item_id']
    
    # Get latest Buff snapshot
    conn = db_client.get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT best_ask, best_bid, sell_order_count, buy_order_count
        FROM buff_snapshots
        WHERE item_id = ?
        ORDER BY timestamp DESC
        LIMIT 1
    """, (item_id,))
    
    buff_row = cursor.fetchone()
    conn.close()
    
    if buff_row and buff_row['sell_order_count'] and buff_row['buy_order_count']:
        ask_count = buff_row['sell_order_count'] or 0
        bid_count = buff_row['buy_order_count'] or 0
        
        if ask_count > 0 or bid_count > 0:
            imbalance_ratio = ask_count / (bid_count + 1)  # +1 to avoid div by zero
            buff_imbalance_data.append({
                'item_id': item_id,
                'market_hash_name': row['market_hash_name'],
                'ask_count': ask_count,
                'bid_count': bid_count,
                'imbalance_ratio': imbalance_ratio,
                'spread_pct': row.get('spread_pct', 0)
            })

df_imbalance = pd.DataFrame(buff_imbalance_data)

if len(df_imbalance) > 0:
    print("Buff Order Book Imbalance Analysis:")
    print(f"Average ask/bid ratio: {df_imbalance['imbalance_ratio'].mean():.2f}")
    print(f"Items with more asks than bids: {(df_imbalance['imbalance_ratio'] > 1).sum()} / {len(df_imbalance)}")
    print(f"\nTop 10 by imbalance ratio (ask-heavy):")
    print(df_imbalance.nlargest(10, 'imbalance_ratio')[['market_hash_name', 'ask_count', 'bid_count', 'imbalance_ratio', 'spread_pct']])
    
    # Scatter: imbalance ratio vs spread
    plt.figure(figsize=(10, 6))
    plt.scatter(df_imbalance['imbalance_ratio'], df_imbalance['spread_pct'], alpha=0.6)
    plt.xlabel('Ask/Bid Ratio (higher = more asks)')
    plt.ylabel('Spread %')
    plt.title('Buff Order Book Imbalance vs Spread')
    plt.axvline(x=1, color='r', linestyle='--', label='Balanced (1:1)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("No Buff order book data available")


In [ ]:
print("Summary Statistics:")
print(f"Total items analyzed: {len(df_clean)}")
print(f"Items with positive PnL: {len(df_positive)}")
print(f"Average PnL (positive only): {df_positive['pnl_now'].mean():.2f}")
print(f"Median PnL (positive only): {df_positive['pnl_now'].median():.2f}")
print(f"Max PnL: {df_positive['pnl_now'].max():.2f}")

if len(df_risk) > 0:
    print(f"\nRisk Analysis (top candidates):")
    print(f"Average probability of positive PnL: {df_risk['prob_positive'].mean():.2f}")
    print(f"Average expected PnL: {df_risk['expected_pnl'].mean():.2f}")
    print(f"Average 95% VaR: {df_risk['var_95'].mean():.2f}")
